In [22]:
import cortex
import numpy as np
from os.path import join
from cortex.options import config
import h5py
import pickle as pkl
from copy import deepcopy
import matplotlib.pyplot as plt

Note: for this to work, need to point the cortex config filestore to the `ds003020/derivative/pycortex-db` directory.
This might look something like `/home/chansingh/mntv1/deep-fMRI/data/ds003020/derivative/pycortex-db/UTS03/anatomicals/`

In [10]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/results/encoding/bert-10__ndel=4/UTS03/'
corrs = np.load(join(results_dir, 'corrs.npz'))['arr_0']
corrs.shape

(95556,)

In [11]:
perc_threshold = 98
perc = np.percentile(corrs, perc_threshold)
idxs = (corrs > perc)
corrs_thresh = deepcopy(corrs)
corrs_thresh[~idxs] = np.nan

In [ ]:
vol = cortex.Volume(corrs, 'UTS03', xfmname='UTS03_auto')
cortex.quickshow(vol, with_rois=True, cmap='PuBu') #, with_curvature=True, with_sulci=True)

In [ ]:
vol = cortex.Volume(corrs_thresh, 'UTS03', xfmname='UTS03_auto')
cortex.quickshow(vol, with_rois=True, cmap='PuBu') #, with_curvature=True, with_sulci=True)

In [2]:
save_dir = '/home/chansingh/mntv1/deep-fMRI/results/linear_models/oct23'
fname = 'coef_moral_stories_bert-10__ndel=4fmri_seed=3.pkl'
# fname = 'coef_moral_stories_bert-10__ndel=4fmri_seed=3.pkl
coef_fname = join(save_dir, fname)
model = pkl.load(open(coef_fname, 'rb'))

In [19]:
corrs.shape

(95556,)

In [18]:
model.best_estimator_.coef_.squeeze().shape

(94610,)

In [14]:
idxs.sum()

94619